In [ ]:
!pip install yfinance
import yfinance as yf
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [10]:
import sys
print(sys.executable)

import sys
print(sys.version)
print(sys.executable)


/Library/Developer/CommandLineTools/usr/bin/python3
3.9.6 (default, Nov 11 2024, 03:15:38) 
[Clang 16.0.0 (clang-1600.0.26.6)]
/Library/Developer/CommandLineTools/usr/bin/python3
